In [1]:
import openai
import numpy as np
from openai import OpenAI
client = OpenAI()

def make_yes_no_prompt(context: str, question: str, response: str) -> str:
    return f"""Context: {context}

Question: {question}

Response: {response}

Based on the given Context and Question, answer this question:

Is the provided Response correct? Answer only Yes or No.

Answer:
    """

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def yes_score_calculation(logits):
    tokens = [logprob.token for logprob in logits]
    logprobs = [logprob.logprob for logprob in logits]
    # Calculate softmax probabilities
    probs = softmax(np.array(logprobs))
    
    # Pre-select tokens with probabilities greater than 0.001
    next_token_dict = {token: prob for token, prob in zip(tokens, probs) if prob > 0.001}
    
    # Get probabilities for 'Yes' and 'No'
    yes_prob = next_token_dict.get("Yes", 0.0)
    no_prob = next_token_dict.get("No", 0.0)
    
    # Calculate and return yes/no confidence score
    yes_score = yes_prob / (yes_prob + no_prob) if yes_prob != 0 or no_prob != 0 else 0.5
    return yes_score


def get_logits_from_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1,  # Get logits for the next token
        logprobs=True,    
        top_logprobs = 5,
        temperature=0  # Set temperature to 0 for deterministic output
    )
    
    logits = response.choices[0].logprobs.content[0].top_logprobs
    return logits



# Example of a correct answer 

In [2]:

context = "You are a skilled assistant to a molecular biologist with expertise in functional genetics."
question = "Is ACTC1 involved in cardiomyocyte function?"
response = "Yes, ACTC1 is involved in cardiomyocyte function."

prompt = make_yes_no_prompt(context, question, response)

logits = get_logits_from_openai(prompt)
yes_score = yes_score_calculation(logits)
print(f"Yes Score: {yes_score}")


Yes Score: 1.0


# Example of a wrong answer


In [3]:
context = "You are a skilled assistant to a molecular biologist with expertise in functional genetics."
question = "Is MC1R involved in cardiomyocyte function?"
response = "Yes, MC1R is involved in cardiomyocyte function."

prompt = make_yes_no_prompt(context, question, response)

# Replace with your OpenAI API key

logits = get_logits_from_openai(prompt)
yes_score = yes_score_calculation(logits)
print(f"Yes Score: {yes_score}")

Yes Score: 0.0
